In [2]:
import pandas as pd  # Import the pandas library for data manipulation and analysis
import numpy as np  # Import the NumPy library for numerical operations
import xgboost as xgb  # Import the XGBoost library for gradient boosting machine learning models
import gradio as gr  # Import the Gradio library for creating user interfaces
from sklearn.preprocessing import StandardScaler  # Import the StandardScaler for data normalization
import joblib  # Import joblib for loading and saving machine learning models

# Load the shear force prediction model
shear_model = xgb.XGBRegressor()  # Initialize an XGBoost regression model for shear force
shear_model.load_model("xgboost_shear.json")  # Load the pre-trained model from the specified JSON file

# Load the drift prediction model
drift_model = xgb.XGBRegressor()  # Initialize an XGBoost regression model for drift
drift_model.load_model("xgboost_drift.json")  # Load the pre-trained model from the specified JSON file

# Load the scaler
scaler = joblib.load('scaler.joblib')  # Load the pre-fitted StandardScaler from the specified file

# Define the prediction function
def predict(D, H_D, Ad, Ap, G, PT, fc, PGA, CAV, PGV, CAD, PGD, Drms, SD, VA):
    # Create a 2D NumPy array with the input parameters for the model
    input_data = np.array([[D, H_D, Ad, Ap, G, PT, fc, PGA, CAV, PGV, CAD, PGD, Drms, SD, VA]])
    
    # Scale the input data using the previously defined scaler
    input_scaled = scaler.transform(input_data)  # Normalize input data to the model's training scale
    
    # Predict shear force using the shear model
    pred_shear = shear_model.predict(input_scaled)  # Get the prediction for shear force
    shear = pred_shear[0]  # Extract the predicted shear force from the prediction array
    
    # Predict drift using the drift model
    pred_drift = drift_model.predict(input_scaled)  # Get the prediction for drift
    drift = pred_drift[0]  # Extract the predicted drift from the prediction array
    
    # Return both predictions
    return shear, drift  # Return the predicted shear and drift values

# Create a Gradio interface for user interaction
iface = gr.Interface(
    fn=predict,  # Specify the function to call for predictions
    inputs=[
        gr.Number(label="Diameter (m)", value=2.1),  # Input for diameter in meters with a default value
        gr.Number(label="Aspect ratio", value=4.5),  # Input for aspect ratio with a default value
        gr.Number(label="Longitudinal reinforcing steel ratio (%)", value=0.001530745),  # Input for steel ratio
        gr.Number(label="PT steel ratio (%)", value=0.005509084),  # Input for post-tensioned steel ratio
        gr.Number(label="Gravity load ratio (%)", value=0.058750712),  # Input for gravity load ratio
        gr.Number(label="Initial PT force ratio (%)", value=0.076873839),  # Input for initial PT force ratio
        gr.Number(label="Concrete cover compressive strength (kPa)", value=26769),  # Input for concrete strength
        gr.Number(label="PGA (g)", value=0.386808),  # Input for peak ground acceleration
        gr.Number(label="CAV (cm/s)", value=1232.5763),  # Input for cumulative absolute velocity
        gr.Number(label="PGV (cm/s)", value=37.143572),  # Input for peak ground velocity
        gr.Number(label="CAD (cm)", value=127.36346),  # Input for cumulative absolute displacement
        gr.Number(label="PGD (cm)", value=4.3132972),  # Input for permanent ground displacement
        gr.Number(label="Drms (cm)", value=2.0829409),  # Input for root mean square displacement
        gr.Number(label="SD (s)", value=9.1),  # Input for duration of shaking
        gr.Number(label="V/A", value=0.097985574)  # Input for the ratio of velocity to acceleration
    ],
    outputs=[
        gr.Number(label="Shear (kN)"),  # Output field for shear force prediction
        gr.Number(label="Drift (%)")  # Output field for drift prediction
    ],
    title="Seismic Response Prediction",  # Title of the Gradio interface
    description="Input parameters for seismic response prediction."  # Description of what the interface does
)

# Launch the Gradio interface for user interaction
iface.launch()  # Start the web application to allow users to input parameters and get predictions

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


c:\Program Files\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
